<a href="https://colab.research.google.com/github/michealman114/GAN-Experiments/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [21]:
"""
things to do for GAN
generator
discriminator

loss
wasserstein loss
loss for discrimnator: d(x) - d(g(r)) where x are real inputs and r is random noise fed into the generator

loss for generator: d(g(r)) 

discriminator tries to maximize the difference between values for generated and real inputs

generator tries to maximize the value of generated inputs from the discriminator


def discrim_loss(results, labels):

  parameters
  loss = average of d(g(r)) - d(x) (negative for minimization)
  results: batch_size * 1 tensor
  labels: batch_size tensor of labels where 1 is real and 0 is generated

  returns:
  1 dimensional tensor containing the loss
  real_labels = labels.view(-1, 1)
  fake_labels = (1 - labels).view(-1,1)
  real = torch.sum(real_labels)
  fake = torch.sum(fake_labels)
  #print(real_labels)
  #print(fake_labels)
  return torch.sum(results * fake_labels) / fake - torch.sum(results * real_labels) / real

  pass

def gen_loss(results, labels):
  labels = 1 - labels
  labels = labels.view(-1, 1)
  gen_labels = torch.sum(labels)
  return torch.sum(results * labels) / -gen_labels
"""

'\nthings to do for GAN\ngenerator\ndiscriminator\n\nloss\nwasserstein loss\nloss for discrimnator: d(x) - d(g(r)) where x are real inputs and r is random noise fed into the generator\n\nloss for generator: d(g(r)) \n\ndiscriminator tries to maximize the difference between values for generated and real inputs\n\ngenerator tries to maximize the value of generated inputs from the discriminator\n\n\ndef discrim_loss(results, labels):\n\n  parameters\n  loss = average of d(g(r)) - d(x) (negative for minimization)\n  results: batch_size * 1 tensor\n  labels: batch_size tensor of labels where 1 is real and 0 is generated\n\n  returns:\n  1 dimensional tensor containing the loss\n  real_labels = labels.view(-1, 1)\n  fake_labels = (1 - labels).view(-1,1)\n  real = torch.sum(real_labels)\n  fake = torch.sum(fake_labels)\n  #print(real_labels)\n  #print(fake_labels)\n  return torch.sum(results * fake_labels) / fake - torch.sum(results * real_labels) / real\n\n  pass\n\ndef gen_loss(results, lab

In [25]:
import torch.nn as nn

class Gen(nn.Module):
  # GAN takes an input of noise of size 10 and constructs a 7 bit number in the form of a 7 element tensor
  def __init__(self):
    super(Gen, self).__init__()
    self.net = nn.Sequential(
        nn.Linear(10,10),
        nn.ReLU(),
        nn.Linear(10,7),
        nn.Sigmoid()
    )
  def forward(self, input):
    return self.net(input)

In [3]:
class Discrim(nn.Module):
  def __init__(self):
    super(Discrim, self).__init__()
    self.net = nn.Sequential(
        nn.Linear(7,7),
        nn.ReLU(),
        nn.Linear(7,1),
        nn.ReLU()
    )
  def forward(self, input):
    return self.net(input)

In [58]:
import random
def generate_real(num):
  inputs = [[int(x) for x in format(2*random.randint(0,63), "07b")] for x in range(num)]
  
  return torch.tensor(inputs)

generate_real(20)

tensor([[0, 0, 0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0, 0, 0],
        [0, 1, 0, 1, 1, 0, 0],
        [1, 1, 0, 0, 0, 1, 0],
        [0, 1, 1, 1, 0, 1, 0],
        [0, 1, 1, 0, 1, 0, 0],
        [0, 0, 1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [1, 1, 0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0, 0, 0],
        [0, 1, 1, 0, 1, 1, 0],
        [1, 1, 1, 0, 1, 1, 0],
        [0, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0]])

In [ ]:
"""
from torch import optim
from sklearn.utils import shuffle
loss_fn = nn.BCELoss()

gen1 = Gen()
discrim1 = Discrim()

def train(num_epochs = 1000):
  gen_optim = optim.Adam(gen1.parameters(), lr = 0.001)
  discrim_optim = optim.Adam(discrim1.parameters(), lr = 0.001)

  for i in range(num_epochs):
    noise = torch.rand(200, 10)
    generated = gen1.forward(noise)
    real = generate_real(200).float()
    #data = torch.cat((generated, real)).float()
    #print("data",data.shape)
    gen_labels = torch.zeros(200,1)
    real_labels = torch.ones(200,1)
    #print("labels", labels.shape)

    #shuffled_data, shuffled_labels = shuffle(data,labels)

    #shuffled_labels = torch.flatten(shuffled_labels)

    discrim_gen_out = discrim1.forward(generated)

    #print(out.shape)

    gen_optim.zero_grad()
    gen_loss = -discrim_gen_out
    gen_loss.backward()
    gen_optim.step()

    discrim_gen_out = discrim1.forward(generated.detach())

    discrim_optim.zero_grad()
    loss2 = loss_fn(discrim_gen_out, gen_labels)
    #loss2.backward()

    discrim_optim.zero_grad()
    real_out = discrim1.forward(real)
    loss3 = loss_fn(real_out, real_labels)

    discrim_training_loss = (loss2 + loss3)/2
    
    discrim_training_loss.backward()
    discrim_optim.step()

    if (i%100 == 0):
      print(i, gen_loss.item(), discrim_training_loss.item())
    
  
train()
"""

In [16]:
# https://discuss.pytorch.org/t/set-constraints-on-parameters-or-layers/23620/2
class WeightClipper(object):
    def __init__(self, c, frequency=5):
        self.frequency = frequency
        self.c = abs(c)

    def __call__(self, module):
        # filter the variables to get the ones you want
        if hasattr(module, 'weight'):
            w = module.weight.data
            w = w.clamp(-self.c,self.c)
            module.weight.data = w

In [63]:
# https://agustinus.kristia.de/techblog/2017/02/04/wasserstein-gan/
# alpha = 0.00005, c = 0.01 (clip values of discrim), n_critic = 5
# hyperparameters from tutorial
# reimplement training loop
# train discrim several times, then train generator

from torch import optim

alpha = 0.00005
c = 0.01
n_critic = 5

generator = Gen()
discriminator = Discrim()

clipper = WeightClipper(c)

one = torch.FloatTensor([1])
m_one = torch.FloatTensor([-1])

def train(num_epochs):
    # alpha = 0.00005, c = 0.01 (clip values of discrim), n_critic = 5
    gen_optim = optim.RMSprop(generator.parameters(), lr = alpha)
    discrim_optim = optim.RMSprop(discriminator.parameters(), lr = alpha)

    for i in range(num_epochs):
        for j in range(n_critic):
            noise = torch.rand(200, 10).float()
            fake = generator.forward(noise)
            real = generate_real(200).float()
            fake_loss = discriminator.forward(fake.detach())
            real_loss = discriminator.forward(real)
            # real - fake
            discrim_optim.zero_grad()
            discrim_loss = torch.mean(real_loss, dim=0) - torch.mean(fake_loss, dim = 0)
            #print(discrim_loss.shape)
            discrim_loss.backward(m_one)
            discrim_optim.step()
            discriminator.apply(clipper)
        # training generator
        noise = torch.rand(200,10).float()
        gen_optim.zero_grad()
        gen_loss = torch.mean(discriminator.forward(generator.forward(noise)), dim = 0)
        gen_loss.backward(one)
        gen_optim.step()
  
train(10000)

In [65]:
test_input = torch.rand(1,10)
print(test_input.numpy())
generator.forward(test_input)

[[0.19349211 0.1651839  0.46215117 0.79894763 0.37445194 0.7852252
  0.15941554 0.1212126  0.2515173  0.82409686]]


tensor([[9.9992e-01, 1.1174e-04, 1.0598e-04, 9.9992e-01, 9.9993e-01, 2.8151e-05,
         9.9996e-01]], grad_fn=<SigmoidBackward0>)

In [ ]:
# tensor([[4.8008e-06, 9.9998e-01, 1.4812e-05, 9.9999e-01, 3.3753e-05, 1.0000e+00, 9.9998e-01]], grad_fn=<SigmoidBackward0>)